## Newton's Forward Interpolation

In [3]:
def comb(n, i):
    num = 1
    den = 1

    for j in range(0, i):
        num *= (n-j)
        den *= j+1
    
    return num/den

def newton_forward_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    h = x[1] - x[0]         # Assuming equidistant x values
    u = (x_pred - x[0])/h

    # Maintaining a column of forward difference table
    # [Move ahead in the table in each iteration]
    fd_col = list(y)
    print(fd_col, "\n")

    y_pred = 0
    for i in range(0, n):
        y_pred += comb(u, i) * fd_col[0]
        for j in range(0, n-i-1):
            fd_col[j] = fd_col[j+1] - fd_col[j]
        print(fd_col[:n-i-1], "\n")
    
    return y_pred

In [5]:
# Set of distint points
x = [1, 1.02, 1.04, 1.06, 1.08]
y = [0.242, 0.2371, 0.2323, 0.2275, 0.2227]

# Find interpolated value for
x_pred = 1.015

newton_forward_interpolation(x, y, x_pred)

[0.242, 0.2371, 0.2323, 0.2275, 0.2227] 

[-0.004899999999999988, -0.004799999999999999, -0.004799999999999999, -0.004799999999999999] 

[9.999999999998899e-05, 0.0, 0.0] 

[-9.999999999998899e-05, 0.0] 

[9.999999999998899e-05] 

[] 



0.23830952148437504

## Newton's Backward Interpolation

In [11]:
def newton_backward_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    h = x[1] - x[0]         # Assuming equidistant x values
    u = (x_pred - x[-1])/h

    # Maintaining a column of (reverse) backward difference table
    # [Move ahead in the table in each iteration]
    rbk_col = y[::-1]   # !!!reverse
    print(rbk_col, "\n")

    y_pred = 0
    for i in range(0, n):
        y_pred += (-1)**i * comb(-u, i) * rbk_col[0]
        for j in range(0, n-i-1):
            rbk_col[j] = rbk_col[j] - rbk_col[j+1]
        print(rbk_col[:n-i-1], "\n")
    
    return y_pred

In [12]:
# Set of distint points
x = [1, 1.05, 1.10, 1.15, 1.20, 1.25]
y = [0.682689, 0.706282, 0.728668, 0.749856, 0.769861, 0.788700]

# Find interpolated value for
x_pred = 1.235

newton_backward_interpolation(x, y, x_pred)

[0.7887, 0.769861, 0.749856, 0.728668, 0.706282, 0.682689] 

[0.01883899999999994, 0.02000500000000005, 0.021187999999999985, 0.022386000000000017, 0.023592999999999975] 

[-0.0011660000000001114, -0.001182999999999934, -0.0011980000000000324, -0.001206999999999958] 

[1.6999999999822712e-05, 1.5000000000098268e-05, 8.999999999925734e-06] 

[1.9999999997244444e-06, 6.000000000172534e-06] 

[-4.0000000004480896e-06] 

[] 



0.7831697570560001

## Newton's Divided Difference Interpolation

In [13]:
def newton_divided_difference_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)

    # Maintaining a column of divided difference table
    # [Move ahead in the table in each iteration]
    dd_col = list(y)
    print(dd_col, "\n")

    x_coef = 1
    y_pred = 0
    for i in range(0, n):
        y_pred += x_coef * dd_col[0]
        x_coef *= (x_pred - x[i])
        for j in range(0, n-i-1):
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(x[j+i+1] - x[j])
        print(dd_col[:n-i-1], "\n")
    
    return y_pred

In [14]:
# Set of distint points
x = [0.5, 1.5, 3.0, 5.0, 6.5, 8.0]
y = [1.625, 5.875, 31.0, 131.0, 282.125, 521.0]

# Find interpolated value for
x_pred = 7

newton_divided_difference_interpolation(x, y, x_pred)

[1.625, 5.875, 31.0, 131.0, 282.125, 521.0] 

[4.25, 16.75, 50.0, 100.75, 159.25] 

[5.0, 9.5, 14.5, 19.5] 

[1.0, 1.0, 1.0] 

[0.0, 0.0] 

[0.0] 

[] 



351.0

## Langrange's Interpolation

In [19]:
def L(n, k):
    """
    Lagrange Coefficient Polynomial
    """
    num = 1
    den = 1
    for i in range(n):
        if i != k:
            num *= (x_pred - x[i])
            den *= (x[k] - x[i])
    return num/den

def lagrange_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    y_pred = 0
    for i in range(0, n):
        y_pred += y[i] * L(n, i)
        print(f"L({n-1}, {i}) = {L(n-1, i)}\n")
    
    return y_pred

In [20]:
# Set of distint points
x = [300, 304, 305, 307]
y = [2.4771, 2.4829, 2.4843, 2.4871]

# Find interpolated value for
x_pred = 301

lagrange_interpolation(x, y, x_pred)

L(3, 0) = 0.6

L(3, 1) = 1.0

L(3, 2) = -0.6

L(3, 3) = 0.2857142857142857



2.4785971428571423

## Hermite Interpolation

Using Newton's divided difference approximation

In [22]:
def double_each_element(l):
    n = len(l)
    l_new = []
    for i in range(n):
        l_new.append(l[i])
        l_new.append(l[i])
    return l_new

def hermite_interpolation(x, y, y_red, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    y_der: list
        y derivative values
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    z = double_each_element(x)
    n = len(z)

    # Maintaining a column of divided difference table
    # [Move ahead in the table in each iteration]
    dd_col = double_each_element(y)
    print(dd_col, "\n")

    for j in range(0, n-1):
        if j%2 == 0:
            dd_col[j] = y_der[j//2]
        else:
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(z[j+1] - z[j])
    print(dd_col[:n-1], "\n")

    z_coef = (x_pred - z[0])
    y_pred = y[0]
    for i in range(1, n):
        y_pred += z_coef * dd_col[0]
        z_coef *= (x_pred - z[i])
        for j in range(0, n-i-1):
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(z[j+i+1] - z[j])
        print(dd_col[:n-i-1], "\n")
    
    return y_pred


In [23]:
# Set of distint points
x = [1.3, 1.6, 1.9]
y = [0.6200860, 0.4554022, 0.2818186]
y_der = [-0.5220232, -0.5698958, -0.5811571]

# Find interpolated value for
x_pred = 1.5

hermite_interpolation(x, y, y_der, x_pred)

[0.620086, 0.620086, 0.4554022, 0.4554022, 0.2818186, 0.2818186] 

[-0.5220232, -0.548946, -0.5698958, -0.5786120000000003, -0.5811571] 

[-0.08974266666666673, -0.06983266666666635, -0.029054000000001336, -0.008483666666665451] 

[0.06636666666666795, 0.0679644444444417, 0.06856777777778632] 

[0.0026629629629562617, 0.0010055555555743558] 

[-0.0027623456789698437] 

[] 



0.5118276938271605

## Curve fitting using Principal of Least Squares

In [3]:
import numpy as np

def output(str, v):
    print(f'{str} = {v} | sum = {np.sum(v)}\n')

def curve_fit(x, y, deg=1):
    """
    Arguments
    ---------
    x: numpy.array
        x values
    y: numpy.array
        y value
    deg: int
        Degree of interpolated polynomial

    Returns
    -------
    np.array:
        Returns coefficients of polynomial
        [Starting from highest degree]
    """
    n = deg+1
    X = np.zeros((n, n))
    Y = np.zeros(n)

    # Details
    # output('x**0', x**0)
    # output('x**1', x**1)
    # output('x**2', x**2)
    # output('x**3', x**3)
    # output('x**4', x**4)
    # output('x**5', x**5)
    # output('x**6', x**6)

    output('y', y)
    # output('yx', y*x)
    # output('yx**2', y*x**2)
    # output('yx**3', y*x**3)

    # Initialize X
    for i in range(0, n):
        for j in range(0, n):
            X[i][j] = np.sum(x**(i+j))
    
    # Initialize Y
    for i in range(0, n):
        Y[i] = np.sum(y*x**(i))

    print(X)
    print(Y)

    A =  np.linalg.solve(X, Y)

    return np.flip(A)

In [4]:
# Degree 2

import numpy as np

x = np.array([4, 4.2, 4.5, 4.7, 5.1, 5.5, 5.9, 6.3, 6.8, 7.1])
y = np.array([102.56, 113.18, 130.11, 142.05, 167.53, 195.14, 224.87, 256.73, 299.50, 326.72])
deg = 3

curve_fit(x, y, deg)

y = [102.56 113.18 130.11 142.05 167.53 195.14 224.87 256.73 299.5  326.72] | sum = 1958.39

[[1.00000000e+01 5.41000000e+01 3.03390000e+02 1.75983100e+03]
 [5.41000000e+01 3.03390000e+02 1.75983100e+03 1.05231207e+04]
 [3.03390000e+02 1.75983100e+03 1.05231207e+04 6.46079775e+04]
 [1.75983100e+03 1.05231207e+04 6.46079775e+04 4.05616744e+05]]
[  1958.39     11366.843    68006.6811  417730.09823]


array([-0.01367456,  6.84557777, -2.37922111,  3.42909438])

In [26]:
f = lambda x: -0.01367456*x**3 +  6.84557777*x**2 - 2.37922111*x + 3.42909438

In [38]:
f(7.1)

326.72792444054

In [25]:
# Degree 1 - linear

import numpy as np

x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = np.array([1.3, 3.5, 4.2, 5.0, 7.0, 8.8, 10.1, 12.5, 13.0, 15.6])

curve_fit(x, y)

array([ 1.53818182, -0.36      ])

## Numerical Differenciation

In [6]:
import numpy as np
from math import inf

def two_point_formula(x, y):
    n = len(x)
    h = x[1] - x[0]
    
    lep = [inf for i in range(n)]
    rep = [inf for i in range(n)]

    for i in range(n):
        if i < n-1:
            lep[i] = (y[i+1] - y[i])/h
        if i > 0:
            rep[i] = (y[i] - y[i-1])/h
    
    print("LEP")
    print(lep)
    print("REP")
    print(rep)

def three_point_formula(x, y):
    n = len(x)
    h = x[1] - x[0]
    
    lep = [inf for i in range(n)]
    mid = [inf for i in range(n)]
    rep = [inf for i in range(n)]

    for i in range(n):
        if i < n-2:
            lep[i] = (-3*y[i] + 4*y[i+1] - y[i+2])/(2*h)
        if i > 1:
            rep[i] = (-3*y[i] + 4*y[i-1] - y[i-2])/(2*-h)
        if i > 0 and i < n-1:
            mid[i] = (-y[i-1] + y[i+1])/(2*h)
    
    print("LEP")
    print(lep)
    print("MID")
    print(mid)
    print("REP")
    print(rep)

def five_point_formula(x, y):
    n = len(x)
    h = x[1] - x[0]
    
    lep = [inf for i in range(n)]
    mid = [inf for i in range(n)]
    rep = [inf for i in range(n)]

    for i in range(n):
        if i < n-4:
            lep[i] = (-25*y[i] + 48*y[i+1] -36*y[i+2] + 16*y[i+3] -3*y[i+4])/(12*h)
        if i > 3:
            rep[i] = (-25*y[i] + 48*y[i-1] -36*y[i-2] + 16*y[i-3] -3*y[i-4])/(12*-h)
        if i > 1 and i < n-2:
            mid[i] = (y[i-2] - 8*y[i-1] + 8*y[i+1] - y[i+2])/(12*h)
    
    print("LEP")
    print(lep)
    print("MID")
    print(mid)
    print("REP")
    print(rep)

def three_point_formula2(x, y):
    n = len(x)
    h = x[1] - x[0]
    
    mid = [inf for i in range(n)]

    for i in range(n):
        if i > 0 and i < n-1:
            mid[i] = (y[i-1] - 2*y[i] + y[i+1])/(h**2)
    
    print("MID")
    print(mid)

In [18]:
import numpy as np

x = np.array([0.2, 0.4, 0.6, 0.8, 1.0])
y = np.array([0.9798652, 0.9177710, 0.808038, 0.6386093, 0.3843735])

two_point_formula(x, y)

LEP
[-0.31047099999999994, -0.5486649999999998, -0.8471434999999999, -1.2711790000000003, inf]
REP
[inf, -0.31047099999999994, -0.5486649999999998, -0.8471434999999999, -1.2711790000000003]


In [19]:
three_point_formula(x, y)

LEP
[-0.19137400000000027, -0.3994257499999995, -0.6351257500000003, inf, inf]
MID
[inf, -0.4295679999999999, -0.6979042499999999, -1.05916125, inf]
REP
[inf, inf, -0.6677619999999992, -0.9963827500000002, -1.4831967500000003]


In [16]:
five_point_formula(x, y)

LEP
[-0.19515070833333367, inf, inf, inf, inf]
MID
[inf, inf, -0.6824174583333331, inf, inf]
REP
[inf, inf, inf, inf, -1.5413672083333336]


In [20]:
three_point_formula2(x, y)

MID
[inf, -1.1909699999999992, -1.4923925000000002, -2.1201775000000014, inf]
